1. Create a flat table per each order (flat – for every order bring every relevant information). For example: users registration, country, category etc. Save it to csv or excel file. PLEASE  validate the data.

2. Read the extra text file that can be find here to your current excel

In [5]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [6]:
db_config = {'user': 'practicum_student',         # username             
                      'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password             
                      'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',       #host URL      
                      'port': 6432,              # connection port             
                      'db': 'data-analyst-web-store'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [7]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

In [95]:

query1 = '''
            select
                *
            from
            (
                select  
                    orders.order_id, orders.date,
                    Registration,
                    campaign.campaign_id, campaign.source,  campaign.start_date as camp_start_date, campaign.end_date as camp_end_date,
                    customer_info.customer_id, customer_info.membership, customer_info.gender,
                    countries.country_id, countries.symbol, countries.country,
                    ab_allocation.ab_test_id, ab_allocation.id, ab_allocation.group,
                    ab_info.description, ab_info.start_date as ab_start_date, ab_info.end_date as ab_end_date,
                    order_details.amount,
                    items.item_id, items.price, items.item,
                    vendors.vendor_id, vendors.vendor,
                    categories.category_id, categories.category, categories.us_ship, categories.other_ship,
                    row_number() over (partition by orders.order_id order BY (select null)) as duprank
                from 
                    orders 
                    left join registration on orders.customer_id = registration.customer_id
                    left join campaign on campaign.campaign_id = registration.campaign_id
                    left join customer_info on customer_info.customer_id = orders.customer_id
                    left join countries on countries.country_id = customer_info.country_id
                    left join ab_allocation on ab_allocation.customer_id = orders.customer_id
                    left join ab_info on ab_info.ab_test_id = ab_allocation.ab_test_id
                    left join order_details on order_details.order_id = orders.order_id
                    left join items on items.item_id = order_details.item_id
                    left join vendors on vendors.vendor_id = items.vendor_id
                    left join categories on categories.category_id = items.category_id
            ) as T
            WHERE duprank = 1 
         '''


df = queryResult(query1)

display('duplicated: ',df[df.duplicated()])
display('na:', df[df.isna().any(axis=1)])
display('no filter: ',df)

'duplicated: '

,order_id,date,registration,campaign_id,source,camp_start_date,camp_end_date,customer_id,membership,gender,...,item_id,price,item,vendor_id,vendor,category_id,category,us_ship,other_ship,duprank


'na:'

,order_id,date,registration,campaign_id,source,camp_start_date,camp_end_date,customer_id,membership,gender,...,item_id,price,item,vendor_id,vendor,category_id,category,us_ship,other_ship,duprank
921,922,2021-03-12,"(56,""2021-02-16 00:00:00"",11)",11,Facebook,2020-11-13,2021-03-10,56,yes,Female,...,41,13.49,Eye Shadow Palette - 15 shades,24,beautyo,10,Beauty Care,9.99,19.99,1
983,984,2021-04-21,"(177,""2021-02-01 00:00:00"",11)",11,Facebook,2020-11-13,2021-03-10,177,yes,Female,...,102,59.99,One Piece Vol.1 1st Edition - JP,9,excellent-shop666,3,Books,9.99,14.99,1
1048,1049,2021-05-26,"(479,""2021-03-12 00:00:00"",14)",14,Google Ads,2021-03-12,2021-08-05,479,no,Male,...,42,320.00,Face Mask,24,beautyo,10,Beauty Care,9.99,19.99,1
1089,1090,2021-06-19,"(197,""2021-06-09 00:00:00"",13)",13,Facebook,2021-04-01,2021-07-12,197,no,Female,...,38,120.00,Dragon Ball Z - Officially Licensed Dragon Bal...,15,pixfab,4,Collectibles,9.99,14.99,1
1109,1110,2021-06-27,"(56,""2021-02-16 00:00:00"",11)",11,Facebook,2020-11-13,2021-03-10,56,yes,Female,...,121,45.99,Roku Streaming Stick With HD/4K/HDR Streaming ...,1,doronk,1,Electronics,19.99,24.99,1
1138,1139,2021-07-10,"(210,""2021-05-28 00:00:00"",14)",14,Google Ads,2021-03-12,2021-08-05,210,yes,Female,...,4,25.99,12/14 inch long Aluminum Pizza Shovel With Lon...,18,magicglobalgadgets17,8,Kitchen,9.99,14.99,1
1157,1158,2021-07-23,"(267,""2021-03-10 00:00:00"",12)",12,YouTube,2021-02-23,2021-06-30,267,no,Female,...,142,72.99,Timberland Brooklyn Alpine Chukka Black Nubuck...,2,zoro12,2,Fashion,9.99,14.99,1
1168,1169,2021-07-31,"(164,""2021-04-30 00:00:00"",12)",12,YouTube,2021-02-23,2021-06-30,164,yes,Male,...,11,19.49,500 Pieces Castle Puzzle,20,trainz,6,Toys & Hobbies,4.99,9.99,1
1178,1179,2021-08-02,"(305,""2021-02-11 00:00:00"",11)",11,Facebook,2020-11-13,2021-03-10,305,yes,Female,...,77,77.50,Lego Classic,13,ignitionline,6,Toys & Hobbies,4.99,9.99,1
1196,1197,2021-08-05,"(68,""2021-08-05 00:00:00"",14)",14,Google Ads,2021-03-12,2021-08-05,68,no,Female,...,161,8.96,Yahtzee,20,trainz,6,Toys & Hobbies,4.99,9.99,1


'no filter: '

,order_id,date,registration,campaign_id,source,camp_start_date,camp_end_date,customer_id,membership,gender,...,item_id,price,item,vendor_id,vendor,category_id,category,us_ship,other_ship,duprank
0,1,2019-10-02,"(214,""2019-06-15 00:00:00"",3)",3,Google Ads,2019-05-30,2019-12-31,214,yes,Male,...,156,13.99,Wilson Official Size NFL Ball American Football,21,vhbw-online,5,Sporting Goods,14.99,19.99,1
1,2,2019-10-02,"(459,""2019-06-29 00:00:00"",2)",2,Facebook,2019-05-30,2019-12-31,459,yes,Female,...,52,84.99,GK61s Mechanical Gaming Keyboard,20,trainz,9,Gaming,9.99,14.99,1
2,3,2019-10-02,"(489,""2019-05-29 00:00:00"",1)",1,YouTube,2019-05-30,2019-12-31,489,no,Male,...,116,21.95,Revlon Super Lusrous 3 piece,24,beautyo,10,Beauty Care,9.99,19.99,1
3,4,2019-10-03,"(127,""2019-05-02 00:00:00"",2)",2,Facebook,2019-05-30,2019-12-31,127,yes,Female,...,30,95.95,Cross Trainer Air Walker Glider Home Gym,12,lumo-shop,5,Sporting Goods,14.99,19.99,1
4,5,2019-10-03,"(174,""2019-09-24 00:00:00"",3)",3,Google Ads,2019-05-30,2019-12-31,174,no,Male,...,136,29.95,Star Wars: the Old Republic - Legends Ser.: Fa...,6,senior pac-man,3,Books,9.99,14.99,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1196,2021-08-05,"(11,""2020-05-06 00:00:00"",9)",9,Google Ads,2020-05-01,2020-09-01,11,no,Male,...,70,21.99,Kids Gamer Tee,3,ozhomedirect,2,Fashion,9.99,14.99,1
1196,1197,2021-08-05,"(68,""2021-08-05 00:00:00"",14)",14,Google Ads,2021-03-12,2021-08-05,68,no,Female,...,161,8.96,Yahtzee,20,trainz,6,Toys & Hobbies,4.99,9.99,1
1197,1198,2021-08-05,"(306,""2021-03-02 00:00:00"",12)",12,YouTube,2021-02-23,2021-06-30,306,no,Male,...,49,28.79,Funko Pop! PAIN Glow In The Dark,7,fullstar,4,Collectibles,9.99,14.99,1
1198,1199,2021-08-05,"(422,""2020-11-15 00:00:00"",10)",10,YouTube,2020-08-12,2020-12-31,422,no,Female,...,139,139.99,The Dangling Man 1st Edition,19,premium247,3,Books,9.99,14.99,1


In [86]:
display(df.columns)
for col in df.columns:
    display('col:',col, df[df[col].isna()])

Index(['order_id', 'date', 'registration', 'campaign_id', 'source',
       'camp_start_date', 'camp_end_date', 'customer_id', 'membership',
       'gender', 'country_id', 'symbol', 'country', 'ab_test_id', 'id',
       'group', 'description', 'ab_start_date', 'ab_end_date', 'amount',
       'item_id', 'price', 'item', 'vendor_id', 'vendor', 'category_id',
       'category', 'us_ship', 'other_ship', 'duprank'],
      dtype='object')

'col:'

'order_id'

,order_id,date,registration,campaign_id,source,camp_start_date,camp_end_date,customer_id,membership,gender,...,item_id,price,item,vendor_id,vendor,category_id,category,us_ship,other_ship,duprank


'col:'

...

'other_ship'

,order_id,date,registration,campaign_id,source,camp_start_date,camp_end_date,customer_id,membership,gender,...,item_id,price,item,vendor_id,vendor,category_id,category,us_ship,other_ship,duprank


'col:'

'duprank'

,order_id,date,registration,campaign_id,source,camp_start_date,camp_end_date,customer_id,membership,gender,...,item_id,price,item,vendor_id,vendor,category_id,category,us_ship,other_ship,duprank


In [92]:
df[df.isna().any(axis=0)].ab_test_id

<ipython-input-92-b31bf264c355>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.isna().any(axis=0)].ab_test_id


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
df.iloc[1157]

In [96]:
query1 = '''
            select
                *
            from
                ab_allocation
         '''


df = queryResult(query1)

display('duplicated: ',df[df.duplicated()])
display('na:', df[df.isna().any(axis=1)])
display('no filter: ',df)

'duplicated: '

,id,customer_id,ab_test_id,group


'na:'

,id,customer_id,ab_test_id,group


'no filter: '

,id,customer_id,ab_test_id,group
0,1,1,1,control
1,2,2,1,test
2,3,3,1,control
3,4,4,1,control
4,5,5,1,test
...,...,...,...,...
1580,1581,472,5,control
1581,1582,474,5,control
1582,1583,480,5,control
1583,1584,481,5,control
